In [ ]:
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
!pip install utils
from utils import *
import random
from spacy.tokenizer import Tokenizer
import warnings
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
warnings.filterwarnings('ignore') 
import string
from sklearn.pipeline import make_pipeline
import xlsxwriter 
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import time
from spacy.lang.tr import Turkish
from sklearn.metrics import classification_report, accuracy_score, make_scorer,precision_score,recall_score,f1_score
from transformers import AutoModel, AutoTokenizer
from glove import Corpus, Glove
import fasttext
import fasttext.util
import nltk 
import torch

import math
import bert
from tensorflow.keras import layers
from transformers import AutoModel, AutoTokenizer
import re
import random

In [ ]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [ ]:
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    
    return accuracy_score(y_true, y_pred) # return accuracy score

In [ ]:
def classification_report_with_precision_score(y_true, y_pred):
    
    return precision_score(y_true, y_pred, average='weighted')

In [ ]:
def classification_report_with_recall_score(y_true, y_pred):
    
    return recall_score(y_true, y_pred,average='weighted')

In [ ]:
def classification_report_with_f1_score(y_true, y_pred):
    
    return f1_score(y_true, y_pred,average='weighted')

In [ ]:
model_pre = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased")
 
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased",do_lower_case=True)


In [ ]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT
    
    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.
    
    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids
        
    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids
    
    
    """
 
    input_ids = []
    attention_masks = []
    encoded_dict = tokenizer.encode_plus(
                        text,                     
                        add_special_tokens = True,
                        max_length = 512,      
                        padding = True,
                        return_attention_mask = True,
                        truncation=True,
                        return_tensors = 'pt',
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
 
    tokens_tensor = torch.cat(input_ids, dim=0)
    segments_tensors = torch.cat(attention_masks, dim=0)
    
    return tokens_tensor, segments_tensors
    
def get_bert_embeddings(tokens_tensor, segments_tensors, model_pre):
    """Get embeddings from an embedding model
    
    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids
    
    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token
    
    """
    
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        outputs = model_pre(tokens_tensor, segments_tensors)
        #print("outputs",outputs)
        # Removing the first hidden state
       # The first state is the input state
        hidden_states = outputs[1]
 
    token_vecs = hidden_states
    # Calculate the average of all ? token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    # Move to the CPU and convert to numpy ndarray.
    sentence_embedding = sentence_embedding.detach().cpu().numpy()
 
    return np.array(sentence_embedding.tolist())


In [ ]:
def feature_extraction_bert(text):
    tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    sentence_embedding = get_bert_embeddings(tokens_tensor, segments_tensors, model_pre)
        
    return sentence_embedding

In [ ]:
y_total= []
y_total=df.duygu.tolist()

X_total = []
for element in tqdm(df.tweets):
    X_total.append(feature_extraction_bert(element))

In [ ]:
# Nested CV with parameter optimization
clf = svm.SVC(kernel='linear')

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for bert/SVM: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for bert/SVM: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for bert/SVM: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for bert/SVM: ",f1.mean())

In [ ]:
# Nested CV with parameter optimization
clf = LogisticRegression()

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for bert/LogisticRegression: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for bert/LogisticRegression: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for bert/LogisticRegression: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for bert/LogisticRegression: ",f1.mean())


In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(10, 3), random_state=1)

accuracy = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_accuracy_score))
print("Accurancy for bert/MLPC: ",accuracy.mean())


precision = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_precision_score))
print("Precision for bert/MLPC: ",precision.mean())


recall = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_recall_score))
print("Recall for bert/MLPC: ",recall.mean())


f1 = cross_val_score(clf, X_total, y_total, cv=10, \
               scoring=make_scorer(classification_report_with_f1_score))
print("F1-Score for bert/MLPC: ",f1.mean())
